# Лабораторная работа 9
# Создание текстового файла большого объема с помощью генераторной функции

#### Присяжнюк Александра

In [4]:
from random import randint, choice

In [6]:
?randint

Signature: randint(a, b)
Docstring:
Return random integer in range [a, b], including both end points.
        
File:      c:\users\user\anaconda3\lib\random.py
Type:      method

In [8]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
f'Код символа: {(c:=randint(*latin_code))}, символ: {chr(c)}'

'Код символа: 99, символ: c'

# Функция process_time из модуля time

In [11]:
from time import process_time

In [13]:
?process_time


Docstring:
process_time() -> float

Process time for profiling: sum of the kernel and user-space CPU time.
Type:      builtin_function_or_method

In [15]:
start = process_time()
[x**2 for x in range(1_000_000)]
end = process_time()
print(f'{end - start} секунд')

0.15625 секунд


# Задание 9.1. Запись данных в текстовый файл

In [18]:
file_name = 'test.txt'
f = open(file_name, 'w')

In [20]:
lines_list = [f'{x}\n' for x in 'test']
lines_list

['t\n', 'e\n', 's\n', 't\n']

In [22]:
f.writelines(lines_list)

In [24]:
f.close()

In [26]:
with open(file_name,'r') as f:
    for line in f:
        print(line, end='')

t
e
s
t


In [28]:
lines_gen_expr = (f'{x}\n' for x in range(10**6))
def lines_gen_fun(number_lines=10**6):
    yield from (f'{x}\n' for x in range(number_lines))
with open(file_name,'w') as f:
    f.writelines(lines_gen_expr)
    f.writelines(lines_gen_fun())
 

In [29]:
def lines_gen_fun(number_lines = 10**6):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
    print(f'{file_size/1024**2} Mb')

with open(file_name,'w') as f:
    f.writelines(lines_gen_fun())


6.569757461547852 Mb


In [30]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun()) 
    end = process_time()
    print(end - start, 'секунд')

6.569757461547852 Mb
1.9375 секунд


In [31]:
def lines_gen_fun(number_lines=10**6, status=False):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)

        # отображение статуса записи в файл в процентах
        if status:
            status_number = x / number_lines * 100
            # \r — возврат курсора в начало строки
            print(f'\r{int(status_number)}%', end='', flush=True)

    print(f'\n {file_size/1024**2} Mb')


In [32]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun(3*10**3, status=True)) 
    end = process_time()
    print(end - start, 'секунд')

99%
 0.013246536254882812 Mb
3.609375 секунд


# Задание 9.2. Генерация строк

In [34]:
latin_code = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
number_words = (10, 20)
number_symbols = (5, 10)

In [35]:
len_word = randint(*number_symbols)
word = ''.join([chr(randint(*latin_code)) for i in range(len_word)])
word

'vajenpupu'

In [37]:
number_code=(ord('0'), ord('9'))
kir_code=(ord('А'), ord('Я'))

skroka=' '.join(''.join([chr(randint(*choice([number_code,kir_code,latin_code]))) for i in range(randint(*number_symbols))]) for j in range(*number_words))

In [38]:
skroka

'y87883ЦaЯ 7Э1oЫs 1rС05m qfwЕq17u УКДС45 КЛ6yr 16frТЪg 646699 c36О2i БО79ДВ521М'

In [40]:
def lines_gen_fun_v2(file_size_mb, type_symbols, number_words, number_symbols):
    size = 0
    limit_bytes = file_size_mb * 1024 ** 2   
    while True:
        line = ' '.join(''.join(chr(randint(*type_symbols)) 
                for _ in range(randint(*number_symbols)))
            for _ in range(randint(*number_words)))
        size += len(line) + 1 
        if size > limit_bytes:
            return
        yield line + '\n'

In [49]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun_v2(1.54,latin_code, number_words, number_symbols )) 
    end = process_time()
    print(end - start, 'секунд')

2.109375 секунд


# Задание 9.3. Запись сгенерированных строк в текстовый файл

In [52]:
def file_gen(file_name: str,
 file_size: int,
 type_symbols: str='latin',
 number_words: (int,int)=(10,20),
 number_symbols: (int,int)=(5,10)):
    size = 0
    limit_bytes = file_size * 1024 ** 2   
    lines=''
    while True:
        line = ' '.join(''.join(chr(randint(*type_symbols)) 
                for _ in range(randint(*number_symbols)))
            for _ in range(randint(*number_words)))
        size += len(line) + 1 
        if size > limit_bytes:
            return
        lines= line + '\n'
        # отображение статуса записи в файл в процентах
        if status:
            status_number = size /limit_bytes * 100
            # \r — возврат курсора в начало строки
            print(f'\r{int(status_number)}%', end='', flush=True)
    print(f'\n {size/1024**2} Mb')


In [64]:
def file_gen(file_name: str,
 file_size: int,
 type_symbols: str='latin',
 number_words: (int,int)=(10,20),
 number_symbols: (int,int)=(5,10)):
    size = 0
    limit_bytes = file_size * 1024 ** 2   
    start = process_time()
    symbol_sets = {
        'latin': [(ord('A'), ord('Z'))],
        'digits': [(ord('0'), ord('9'))],
        'cyrillic': [(ord('А'), ord('Я'))]
    }
    last_percent = -1
    with open(file_name,'w') as f:
        for line in lines_gen_fun_v2(file_size,*symbol_sets[type_symbols], number_words, number_symbols):
            size+=len(line)
            f.write(line)
            status_number = int(size /limit_bytes * 100)
            if status_number>last_percent:
                print(f'\r{status_number}%', end='', flush=True)
                last_percent=status_number
    end = process_time()
    print(end - start, 'секунд')
    print(f'\n {size/1024**2} Mb')


In [66]:
file_gen(file_name, 2, 'latin')

2.8125 секунд

 1.9999914169311523 Mb
